# Evolutionary Algorithm approach for Job shop scheduling

In [ ]:
from deap import algorithms, base, creator, tools
import numpy as np
import pandas as pd
import random
from collections import defaultdict

# Load data

In [ ]:
"""
Place the JSP_dataset spreadsheet in the same directory
"""
pathname = 'JSP_dataset.xlsx'
processing_times = pd.read_excel(pathname, sheet_name="Processing Time", index_col=[0])
machine_sequence = pd.read_excel(pathname, sheet_name="Machines Sequence", index_col=[0])
num_machines = processing_times.shape[1] # number of machines
num_jobs = processing_times.shape[0] # number of jobs
num_gene = num_machines * num_jobs # number of genes in a chromosome
processing_times = processing_times.to_numpy().astype(int)

# Set algorithm parameters

In [ ]:
population_size = 30
crossover_probability = 0.8
mutation_probability = 0.2
mutation_selection_rate = 0.2
num_iteration = 1000000
random.seed(42)

# Convert individual to job orders per machine.

In [ ]:
def decode_individual(individual):
    """
    Args:
        individual (list): A list of integers representing a candidate solution.

    Returns:
        list: A list of lists representing the job orders for each machine.
    """
    # Convert individual to job orders per machine
    orders = [[] for _ in range(num_machines)]
    for i in range(num_jobs * num_machines):
        job_index = individual[i]
        machine_index = i % num_machines
        orders[machine_index].append(job_index)
    return orders

# Calculate completion times for each job on each machine and return makespan as fitness.

In [ ]:
def evaluation(individual):
    """
    Args:
        individual (list): A list of integers representing a candidate solution.

    Returns:
        tuple: A tuple containing the makespan as a float and an empty tuple.
    """
    # Calculate completion times for each job on each machine
    orders = decode_individual(individual)
    machine_times = np.zeros(num_machines)
    job_times = np.zeros((num_jobs, num_machines))
    for machine_index in range(num_machines):
        for job_index in orders[machine_index]:
            processing_time = processing_times[job_index][machine_index]
            machine_times[machine_index] = max(machine_times[machine_index], job_times[job_index][machine_index - 1] if machine_index > 0 else 0) + processing_time
            job_times[job_index][machine_index] = machine_times[machine_index]
    
    # Calculate makespan and return as fitness
    makespan = max(machine_times)
    return makespan,


# Fixes the offspring by ensuring that there are no more or less than num_machines in each job sequence.


In [ ]:
def fix_offspring(individual):
    """
    Args:
        individual (list): The offspring to be fixed.

    Returns:
        list: The fixed offspring.
    """
    job_count = defaultdict(int)
    machine_count = defaultdict(int)
    for gene in individual:
        job = gene // num_machines
        machine = gene % num_machines
        job_count[job] += 1
        machine_count[machine] += 1
    
    excess_jobs = [job for job, count in job_count.items() if count > num_machines]
    missing_jobs = [job for job, count in job_count.items() if count < num_machines]
    
    for job in excess_jobs:
        for machine in range(num_machines):
            if job * num_machines + machine not in individual:
                for i in range(len(individual)):
                    if individual[i] % num_machines == machine:
                        individual[i] = job * num_machines + machine
                        break
    for job in missing_jobs:
        job_machines = [machine for machine in range(num_machines) if job * num_machines + machine not in individual]
        for i in range(len(individual)):
            if machine_count[individual[i] % num_machines] < num_jobs and individual[i] // num_machines not in job_machines:
                individual[i] = job * num_machines + individual[i] % num_machines
                break
        else:
            machine = job_machines[0]
            individual.append(job * num_machines + machine)
    return individual

# Generates a custom permutation of job indices based on the given size.

In [ ]:
def custom_permutation(ind_size):
    """
    Args:
        ind_size: The size of the permutation to generate.

    Returns:
        A list of job indices generated by permuting a list of integers
        of size `ind_size`, and then taking the modulus `num_jobs` of each
        element in the resulting list.
    """
    temp = np.random.permutation(ind_size)
    res = []
    for i in range(len(temp)):
        res.append(temp[i] % num_jobs)
    return res

# Create DEAP toolbox and register functions

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
toolbox.register("indices", custom_permutation, num_gene)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual, n=population_size)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=mutation_selection_rate)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluation)

# Run algorithm

In [ ]:
population = toolbox.population()
offspring = algorithms.varAnd(population, toolbox, crossover_probability, mutation_probability)

best_fitness = float('inf')
no_improvement_count = 0
early_stopping_threshold = 1000
makespans = set()
makespans_record = []
for generation in range(num_iteration):
    # Evaluate fitness of offspring
    fitnesses = list(map(toolbox.evaluate, offspring))
    for individual, fitness in zip(offspring, fitnesses):
        individual.fitness.values = fitness
        
    # Select the next generation's population
    population = toolbox.select(offspring, k=population_size)
    
    # Apply crossover and mutation to create the offspring
    offspring = algorithms.varAnd(population, toolbox, crossover_probability, mutation_probability)
    
    # Fix offspring by replacing excess jobs and adding missing jobs
    for i in range(len(offspring)):
        offspring[i] = fix_offspring(offspring[i])
    
    # Update statistics and print progress
    fits = [ind.fitness.values[0] for ind in population]
    min_fit = min(fits)
    if min_fit < best_fitness:
        best_fitness = min_fit
        best_individual = population[fits.index(min_fit)]
        no_improvement_count = 0
    else:
        no_improvement_count += 1
        if no_improvement_count > early_stopping_threshold:
            print(f"Stopping early after {generation} generations with no improvement.")
            print(f"Optimal Sequence: {best_individual}")
            break
    makespans.add(best_fitness)
    makespans_record.append(best_fitness)
    print(f"Generation {generation + 1}: Min fitness = {best_fitness}")

In [ ]:
print(makespans)
# print(makespans_record)
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot([i for i in range(len(makespans_record))],makespans_record,'b')
plt.ylabel('makespan',fontsize=15)
plt.xlabel('generation',fontsize=15)
plt.show()